In [2]:
import itertools
import pandas as pd
import numpy as np
import random
import csv
import time

import matplotlib.pyplot as plt

import tensorflow as tf

import keras.backend as K
from keras import Sequential
from keras.layers import Dense, Dropout

In [3]:
pd.set_option('display.max_row', 1000)

# To do

## [1] 데이터 전처리

+ (저희 전에 얘기했던 중복값 처리) + (usercount >= 20)[O]  + (itemcount(아이템당 팔린 수)>5 (O))

+ embedding 할 때는 + ( rating >3 )

## [2] 중복 이슈

+ 같은 user, 같은 item => 하나만 남기고 중복 처리  
    + rating: 평균 or 최근 or 다른 방법
    + timestamp : 최근
    


-------------------------------------
## [1] 데이터 전처리

### Amazon Books 데이터 불러 오기 &  user,item int로 바꾸기

In [4]:
data = pd.read_csv('./data/Books.csv', names=['item', 'user', 'rating', 'timestamp'])

In [5]:
# User: str to int
data.user = pd.Categorical(data.user)
data['user'] = data.user.cat.codes

# Item: str to int
data.item = pd.Categorical(data.item)
data['item'] = data.item.cat.codes

data.head()

,item,user,rating,timestamp
0,122,1376859,5.0,1123804800
1,122,3094405,5.0,1112140800
2,122,3923726,5.0,1081036800
3,122,3527298,5.0,1077321600
4,122,8042828,5.0,1475452800


### [1-1] user history >= 20

In [6]:
a = pd.DataFrame(data['user'].value_counts())
user_indices = a[a['user']>=20].index 

data_20 = data[data['user'].isin(user_indices)]

In [9]:
data_20.value_counts('user')

user
6831279     9684
5775864     9074
8373854     7077
13318960    5842
3752841     4437
            ... 
8468147       20
14720415      20
11296946      20
14720316      20
2386433       20
Length: 301567, dtype: int64

In [8]:
data_20.to_csv('./data/books_20.csv', index=False, header=None)

In [3]:
data_20 = pd.read_csv('./data/books_int_20.csv', names=['item', 'user', 'rating', 'timestamp'], 
)

In [11]:
data_20.shape

(15731887, 4)

In [51]:
def history_upper20(data):
    a = pd.DataFrame(data['user'].value_counts())
    user_indices = a[a['user']>=20].index 
    data_20 = data[data['user'].isin(user_indices)]
    
    return data_20

In [10]:
data_20 = data_20[['user','item','rating','timestamp']]

### [1-2] item count >= 5

In [39]:
item_counts = pd.DataFrame(data['item'].value_counts())
item_indices = item_counts[item_counts['item']>=5].index 

data_5 = data[data['item'].isin(item_indices)]

In [40]:
data_5.value_counts('item')

item
288015     58150
168974     44956
5794       44381
100968     37783
205270     36620
           ...  
2428001        5
946143         5
946129         5
2249729        5
1452684        5
Length: 1163015, dtype: int64

In [41]:
data_5.to_csv('./data/books_5.csv', index=False, header=None)

In [50]:
def item_upper5(data):
    item_counts = pd.DataFrame(data['item'].value_counts())
    item_indices = item_counts[item_counts['item']>=5].index 
    data_5 = data[data['item'].isin(item_indices)]
    return data_5

### [1-3] rating > 3

In [44]:
rating_indices = data['rating']>3
data_3 = data[rating_indices]

In [45]:
data_3.value_counts('rating')

rating
5.0    33979736
4.0     9557650
dtype: int64

In [49]:
def rating_upper3(data):
    rating_indices = data['rating']>3
    data_3 = data[rating_indices]
    return data_3

## 전처리 이슈 : 전처리 적용 순서

In [53]:
data_20 = history_upper20(data)
data_20_5 = item_upper5(data_20)

In [55]:
print(data_20_5.value_counts('user'),data_20_5.value_counts('item'))

user
5775864     8760
13318960    5778
6831279     5011
8373854     4638
2213957     3813
            ... 
13069247       1
7859           1
3804230        1
9040673        1
1716448        1
Length: 301436, dtype: int64 item
288015     8799
168974     7078
5794       6661
2695054    6294
70403      5738
           ... 
870448        5
870454        5
870766        5
870807        5
12            5
Length: 456236, dtype: int64


In [56]:
data_5 = item_upper5(data)
data_5_20 = history_upper20(data_5)

In [57]:
print(data_5_20.value_counts('user'),data_5_20.value_counts('item'))

user
5775864     8992
6831279     6617
8373854     6164
13318960    5814
1476169     3887
            ... 
13488146      20
9400277       20
9400507       20
3283481       20
7675302       20
Length: 278061, dtype: int64 item
288015     8626
168974     6934
5794       6558
2695054    6162
70403      5603
           ... 
1421515       1
1421517       1
1421523       1
1421525       1
1251907       1
Length: 957809, dtype: int64


+ 위와 같이 전처리 적용 순서에 따라 데이터가 달라지는데 어떤 순서로 적용해야하는지

---------------------------
## [2] 같은 user,같은 item 중복 제거
### [2-1] timestamp : 최신, rating : 가장 최신거의 rating 택

In [12]:
sorted_data = data_20.sort_values(['user','item','timestamp'],ascending=[True,True,True])

In [23]:
sorted_data.head(20)

,user,item,rating,timestamp
40317784,94,463950,4.0,1471478400
40432628,94,494004,4.0,1466208000
40450458,94,499324,5.0,1473120000
40452627,94,499552,5.0,1466380800
47221368,94,503111,3.0,1485475200
13803790,94,507879,4.0,1437696000
13860378,94,509947,5.0,1437782400
36666629,94,568768,4.0,1471132800
36775888,94,569391,4.0,1434844800
40765175,94,572009,4.0,1478304000


In [7]:
grouped_data = data_20.groupby(['user','item'])['timestamp'].max()

+ 같은 item, 다른 timestamp 예시 준비

In [13]:
sorted_data[sorted_data['user'] == 6831279][sorted_data['item'] == 303811]

/home/aiffel0044/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,user,item,rating,timestamp
9953444,6831279,303811,5.0,1487548800
9953445,6831279,303811,5.0,1487548800
9953446,6831279,303811,5.0,1487548800
9953447,6831279,303811,5.0,1487548800
9953448,6831279,303811,5.0,1487548800
9953442,6831279,303811,5.0,1487635200
9953443,6831279,303811,5.0,1487635200
9953408,6831279,303811,5.0,1487721600
9953439,6831279,303811,5.0,1487721600
9953440,6831279,303811,5.0,1487721600


In [14]:
latest_data = sorted_data.drop_duplicates(['user','item'],keep='last')

+ 잘 적용되었나 확인

In [15]:
latest_data[latest_data['user']==6831279][latest_data['item']==303811]

/home/aiffel0044/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,user,item,rating,timestamp
9953403,6831279,303811,5.0,1487894400


In [16]:
latest_data.shape

(15611592, 4)

+ 파일 저장

In [17]:
latest_data.to_csv('./data/latest_data.csv', index=False, header=None)

### [2-2] timestamp : 최신, rating : 평균 rating
+ 이 경우 timestamp를 user,item이 같은 timestamp들 중에서 어떤걸로 선택해야할지?
+ 여기서는 일단 최신 timestamp를 선택하는걸로 진행함

In [24]:
sorted_data['mean_rating'] = sorted_data.groupby(['user','item'])['rating'].transform('mean')

+ user,item 쌍으로 그룹을 나눠준 뒤, 그룹별 rating들의 평균값을 구한 뒤 mean_rating이라는 열에 추가

In [25]:
sorted_data.head(10)

,user,item,rating,timestamp,mean_rating
40317784,94,463950,4.0,1471478400,4.0
40432628,94,494004,4.0,1466208000,4.0
40450458,94,499324,5.0,1473120000,5.0
40452627,94,499552,5.0,1466380800,5.0
47221368,94,503111,3.0,1485475200,3.0
13803790,94,507879,4.0,1437696000,4.0
13860378,94,509947,5.0,1437782400,5.0
36666629,94,568768,4.0,1471132800,4.0
36775888,94,569391,4.0,1434844800,4.0
40765175,94,572009,4.0,1478304000,4.0


+ rating 평균이 잘 구해졌나 확인

In [26]:
sorted_data[sorted_data['user'] == 6831279][sorted_data['item']== 292585]

/home/aiffel0044/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,user,item,rating,timestamp,mean_rating
9723975,6831279,292585,4.0,1459728000,4.142857
9723976,6831279,292585,4.0,1459728000,4.142857
9723977,6831279,292585,4.0,1459728000,4.142857
9723978,6831279,292585,4.0,1459728000,4.142857
9723979,6831279,292585,4.0,1459728000,4.142857
9724016,6831279,292585,4.0,1459728000,4.142857
9724017,6831279,292585,4.0,1459728000,4.142857
9724018,6831279,292585,4.0,1459728000,4.142857
9724019,6831279,292585,4.0,1459728000,4.142857
9724020,6831279,292585,5.0,1459728000,4.142857


+ 잘 구해진거 확인했으니 중복 삭제한 후 , rating 칼럼 삭제, mean_rating 칼럼명 rating으로 바꾸기

In [27]:
dropped_data = sorted_data.drop_duplicates(['user','item'],keep='last')

In [28]:
# drop된거 확인
dropped_data[dropped_data['user'] == 6831279][dropped_data['item']== 292585]

/home/aiffel0044/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,user,item,rating,timestamp,mean_rating
9724024,6831279,292585,4.0,1459728000,4.142857


In [29]:
# 기존 rating 열 삭제
dropped_data = dropped_data.drop(columns='rating')

In [30]:
# rating 열 삭제 확인
dropped_data[dropped_data['user'] == 6831279][dropped_data['item']== 292585]

/home/aiffel0044/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,user,item,timestamp,mean_rating
9724024,6831279,292585,1459728000,4.142857


In [32]:
# mean_rating 열 이름 rating으로 수정
dropped_data.rename(columns={"mean_rating":"rating"}, inplace = True)

In [33]:
dropped_data.head(10)

,user,item,timestamp,rating
40317784,94,463950,1471478400,4.0
40432628,94,494004,1466208000,4.0
40450458,94,499324,1473120000,5.0
40452627,94,499552,1466380800,5.0
47221368,94,503111,1485475200,3.0
13803790,94,507879,1437696000,4.0
13860378,94,509947,1437782400,5.0
36666629,94,568768,1471132800,4.0
36775888,94,569391,1434844800,4.0
40765175,94,572009,1478304000,4.0


In [34]:
# 원래 열 순서로 수정
dropped_data = dropped_data[['user','item','rating','timestamp']]

In [36]:
# 최종 확인
dropped_data[dropped_data['user'] == 6831279][dropped_data['item']== 292585]

/home/aiffel0044/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,user,item,rating,timestamp
9724024,6831279,292585,4.142857,1459728000


In [37]:
# 파일 저장
dropped_data.to_csv('./data/latest_mean_rating.csv', index=False, header=None)

-------------------------------
## 이슈 총 정리

1. 전처리 적용 순서
2. rating을 평균으로 적용 시 timestamp는 그 중 어떤 것으로 선택해야할지 ?